In [34]:
from sortedcontainers import SortedSet

class CSP:
    def __init__(self):
        
        #case 1
        self.variables = ["PP1-Mon", "PP1-Wed", "AI-Tue", "AI-Fri", "TOC-Wed", "TOC-Fri", "ML-Tue", "ML-Fri", "MD-Thu", "MD-Sat"]
        self.domains = {"PP1-Mon": ["A", "C"], "PP1-Wed": ["A", "C"], "AI-Tue": ["B", "C"], "AI-Fri": ["B", "C"], "TOC-Wed": ["A"], "TOC-Fri": ["A"], "ML-Tue": ["B", "C"], "ML-Fri": ["B", "C"], "MD-Thu": ["A", "B"], "MD-Sat": ["A", "B"]}
        self.neighbors = {"PP1-Mon": [], "PP1-Wed": ["TOC-Wed"], "AI-Tue": ["ML-Tue"], "AI-Fri": ["ML-Fri"], "TOC-Wed": ["PP1-Wed"], "TOC-Fri": [], "ML-Tue": ["AI-Tue"], "ML-Fri": ["AI-Fri"], "MD-Thu": [], "MD-Sat": []}
        self.current_domains = {"PP1-Mon": ["A", "C"], "PP1-Wed": ["A", "C"], "AI-Tue": ["B", "C"], "AI-Fri": ["B", "C"], "TOC-Wed": ["A"], "TOC-Fri": ["A"], "ML-Tue": ["B", "C"], "ML-Fri": ["B", "C"], "MD-Thu": ["A", "B"], "MD-Sat": ["A", "B"]}
       
        self.nassigns = 0
        
        #case 2
        #self.variables = ["PP1", "AI", "TOC", "ML", "MD"]
        #self.domains = {"PP1": ["A", "C"], "AI": ["B", "C"] , "TOC": ["A"], "ML": ["B", "C"] , "MD": ["A", "B"]}
        #self.neighbors = {"PP1": ["TOC"], "AI": ["ML"] , "TOC": ["PP1"], "ML": ["AI"] , "MD": []}
        #self.current_domains = {"PP1": ["A", "C"], "AI": ["B", "C"] , "TOC": ["A"], "ML": ["B", "C"] , "MD": ["A", "B"]}
    
    def assign(self, var, val, assignment):
        """Add {var: val} to assignment; Discard the old value if any."""
        assignment[var] = val
        self.nassigns += 1

    def unassign(self, var, assignment):
        """Remove {var: val} from assignment.
        DO NOT call this if you are changing a variable to a new value;
        just call assign for that."""
        if var in assignment:
            del assignment[var]

    def nconflicts(self, var, val, assignment):
        """Return the number of conflicts var=val has with other variables."""

        # Subclasses may implement this more efficiently
        def conflict(var2):
            return var2 in assignment and val == assignment[var2]
        i=0
        for v in self.neighbors[var]:
            if conflict(v):
                i +=1
        return i

    def display(self, assignment):
        """Show a human-readable representation of the CSP."""
        # Subclasses can print in a prettier way, or display with a GUI
        print(assignment)

    # These methods are for the tree and graph-search interface:

    def actions(self, state):
        """Return a list of applicable actions: non conflicting
        assignments to an unassigned variable."""
        if len(state) == len(self.variables):
            return []
        else:
            assignment = dict(state)
            var = first([v for v in self.variables if v not in assignment])
            return [(var, val) for val in self.domains[var]
                    if self.nconflicts(var, val, assignment) == 0]

    def result(self, state, action):
        """Perform an action and return the new state."""
        (var, val) = action
        return state + ((var, val),)

    def goal_test(self, state):
        """The goal is to assign all variables, with all constraints satisfied."""
        assignment = dict(state)
        return (len(assignment) == len(self.variables)
                and all(self.nconflicts(variables, assignment[variables], assignment) == 0
                        for variables in self.variables))

    # These are for constraint propagation

    def suppose(self, var, value):
        """Start accumulating inferences from assuming var=value."""
        removals = [(var, a) for a in self.current_domains[var] if a != value]
        self.current_domains[var] = [value]
        return removals

    def prune(self, var, value, removals):
        """Rule out var=value."""
        self.current_domains[var].remove(value)
        if removals is not None:
            removals.append((var, value))


    def restore(self, removals):
        """Undo a supposition and all inferences from it."""
        for B, b in removals:
            self.current_domains[B].append(b)

    # Variable ordering

    def mini_remaining_val(self, assignment):
        """Minimum-remaining-values heuristic."""
        minimum=1000
        var=""
        for v in self.variables:
            if v not in assignment:
                if len(self.current_domains[v]) < minimum:
                    minimum = len(self.current_domains[v])
                    var = v
            
        return var



    # Value ordering


    def least_constrained_val(self, var, assignment):
        """Least-constraining-values heuristic."""

        def counter_constraints(var, val):
            count=0
            for var2 in self.neighbors[var]:
                for value in self.current_domains[var2]:
                    if value!= val:
                        count += 1
            return count
                        
        return sorted(self.current_domains[var], key=lambda val: counter_constraints(var, val))



    # Inference


    def forward_checking(self, var, value, assignment, removals):
        """Prune neighbor values inconsistent with var=value."""
        for B in self.neighbors[var]:
            if B not in assignment:
                for b in self.current_domains[B][:]:
                    if value == b:
                        self.prune(B, b, removals)
                if not self.current_domains[B]:
                    return False
        return True






# The search

def backtracking_search(csp):

    def backtrack(assignment):
        if len(assignment) == len(csp.variables):
            return assignment
        var = csp.mini_remaining_val(assignment)
        for value in csp.least_constrained_val(var, assignment):
            if 0 == csp.nconflicts(var, value, assignment):
                csp.assign(var, value, assignment)
                removals = csp.suppose(var, value)
                if csp.forward_checking(var, value, assignment, removals):
                    result = backtrack(assignment)
                    if result is not None:
                        return result
                csp.restore(removals)
        csp.unassign(var, assignment)
        return None

    result = backtrack({})
    assert result is None or csp.goal_test(result)
    return result


csp = CSP()

backtracking_search(csp)


{'TOC-Wed': 'A',
 'PP1-Wed': 'C',
 'TOC-Fri': 'A',
 'PP1-Mon': 'A',
 'AI-Tue': 'B',
 'ML-Tue': 'C',
 'AI-Fri': 'B',
 'ML-Fri': 'C',
 'MD-Thu': 'A',
 'MD-Sat': 'A'}